<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/RAG_Restaurant_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU \
    langchain-pinecone\
    langchain-text-splitters \
    langchain_groq\
    langchain \
    pinecone-notebooks\
    gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.9/321.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3

In [2]:
!pip install chromadb langchain langchainhub langchain-community transformers sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.7 MB/s eta 0:00

In [3]:
!pip install --upgrade pydantic

In [2]:
import os
import time
from langchain_text_splitters import MarkdownHeaderTextSplitter
from langchain_pinecone import PineconeEmbeddings, PineconeVectorStore
from langchain.chains import RetrievalQA
from pinecone import Pinecone, ServerlessSpec
import gradio as gr
from langchain_groq import ChatGroq

In [3]:
from google.colab import userdata
# available at platform.openai.com/api-keys
os.environ['GROQ_API_KEY'] = userdata.get('groq_api_key')

In [4]:
import os

# initialize connection to pinecone (orget API key at app.pinecone.io)
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

api_key = os.environ.get("PINECONE_API_KEY")

In [5]:
from langchain_text_splitters import MarkdownHeaderTextSplitter


markdown_document = """
# Bella Vista Italian Restaurant Menu

## Appetizers (Antipasti)

1. Bruschetta al Pomodoro - $12
   Toasted artisanal bread topped with a blend of ripe Roma tomatoes, fresh basil, and extra virgin olive oil. Finished with a drizzle of aged balsamic vinegar.

2. Caprese Salad - $14
   Slices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves. Drizzled with extra virgin olive oil and 12-year-aged balsamic glaze.

3. Calamari Fritti - $16
   Tender rings of calamari lightly dusted in seasoned flour and flash-fried to golden perfection. Served with our house-made spicy marinara sauce and lemon wedges.

## Main Courses (Primi Piatti)

4. Spaghetti alla Carbonara - $22
   Al dente spaghetti tossed in a creamy sauce made with farm-fresh eggs, pecorino Romano cheese, and crispy pancetta. Finished with freshly cracked black pepper and parsley.

5. Risotto ai Funghi Porcini - $26
   Arborio rice slowly cooked to creamy perfection with premium dried porcini mushrooms, white wine, and Parmigiano-Reggiano. Garnished with fresh thyme and truffle oil.

6. Lasagna alla Bolognese - $24
   Layers of fresh pasta sheets, rich meat ragù made with ground beef and pork, creamy béchamel sauce, and melted mozzarella. Baked until golden and bubbly.

## Second Courses (Secondi Piatti)

7. Osso Buco alla Milanese - $32
   Slow-braised veal shank in white wine and vegetables, topped with gremolata. Served with saffron risotto and roasted vegetables.

8. Branzino al Forno - $30
   Whole Mediterranean sea bass stuffed with lemon and herbs, oven-roasted and filleted tableside. Served with roasted potatoes and grilled asparagus.

9. Pollo al Marsala - $28
   Tender chicken breast sautéed with mushrooms in a rich Marsala wine sauce. Served with creamy polenta and seasonal vegetables.

## Desserts (Dolci)

10. Tiramisu - $12
    Layers of coffee-soaked ladyfingers and creamy mascarpone cheese, dusted with premium cocoa powder. Served with a shot of espresso on the side.

11. Panna Cotta - $10
    Silky vanilla bean infused cream custard, topped with a mixed berry compote and fresh mint.

12. Cannoli Siciliani - $9
    Two crispy pastry tubes filled with sweet ricotta cream, chocolate chips, and candied orange peel. Dusted with powdered sugar and served with pistachio gelato.

## Beverages (Bevande)

13. Espresso - $3.50
    Single shot of our premium Italian roast coffee

14. Cappuccino - $5
    Espresso topped with steamed milk and foam

15. Italian Sodas - $4.50
    Choice of flavors: Blood Orange, Lemon, or Raspberry

16. House Red Wine (glass) - $9 / (bottle) - $36
    Montepulciano d'Abruzzo - Medium-bodied with notes of cherry and spice

17. House White Wine (glass) - $8 / (bottle) - $32
    Pinot Grigio - Crisp and refreshing with hints of pear and citrus

Our dishes are prepared with the finest ingredients, including imported Italian products and locally sourced organic produce. Gluten-free and vegetarian options are available upon request. Buon appetito!
"""

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2")
]

markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on, strip_headers=False
)
md_header_splits = markdown_splitter.split_text(markdown_document)

print(md_header_splits)

[Document(metadata={'Header 1': 'Bella Vista Italian Restaurant Menu', 'Header 2': 'Appetizers (Antipasti)'}, page_content='# Bella Vista Italian Restaurant Menu  \n## Appetizers (Antipasti)  \n1. Bruschetta al Pomodoro - $12\nToasted artisanal bread topped with a blend of ripe Roma tomatoes, fresh basil, and extra virgin olive oil. Finished with a drizzle of aged balsamic vinegar.  \n2. Caprese Salad - $14\nSlices of creamy buffalo mozzarella layered with vine-ripened tomatoes and fresh basil leaves. Drizzled with extra virgin olive oil and 12-year-aged balsamic glaze.  \n3. Calamari Fritti - $16\nTender rings of calamari lightly dusted in seasoned flour and flash-fried to golden perfection. Served with our house-made spicy marinara sauce and lemon wedges.'), Document(metadata={'Header 1': 'Bella Vista Italian Restaurant Menu', 'Header 2': 'Main Courses (Primi Piatti)'}, page_content='## Main Courses (Primi Piatti)  \n4. Spaghetti alla Carbonara - $22\nAl dente spaghetti tossed in a c

In [26]:
import os
import time
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

# Initialize Hugging Face Embeddings
embedding_model = "intfloat/multilingual-e5-large"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)

# Define ChromaDB storage path (can be in-memory by setting persist_directory=None)
chroma_persist_directory = "./chroma_db"

# Load or create ChromaDB vector store
docsearch = Chroma.from_documents(
    documents=md_header_splits,  # Replace with your actual document splits
    embedding=embeddings,
    persist_directory=chroma_persist_directory
)

# Initialize LLM
llm = ChatGroq(
    model_name='deepseek-r1-distill-llama-70b',
)

# Define retrieval-based QA chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)


In [27]:
query1 = "What are the appetizer options available at Bella Vista Italian Restaurant?"

query2 = "What vegetarian main course options are available, and can you describe one in detail without missing its price?"

In [28]:
query1_with_knowledge = qa.invoke(query1)
query1_without_knowledge = llm.invoke(query1)

print(query1_with_knowledge['result'])
print()

<think>
Okay, I need to figure out the appetizer options at Bella Vista Italian Restaurant based on the provided context. Let me start by looking through the given menu sections.

First, I see two sections labeled "Appetizers (Antipasti)" with items numbered 1 through 3. Each item has a name, description, and price. Let me list them out.

1. Bruschetta al Pomodoro is $12. It's toasted artisanal bread with tomatoes, basil, and olive oil, finished with balsamic vinegar. That sounds like a classic appetizer.

2. Caprese Salad is $14. It has mozzarella, tomatoes, basil, with olive oil and balsamic glaze. I know Caprese is a traditional Italian salad, so that fits as an appetizer.

3. Calamari Fritti is $16. It's fried calamari with spicy marinara sauce and lemon. Calamari is a common appetizer in Italian restaurants, so that makes sense.

I also see a "Beverages (Bevande)" section with items 13 to 17, but those are drinks, so they're not appetizers. The user specifically asked for appetize

In [29]:
print(query1_without_knowledge.content)

<think>

</think>

As an AI, I don't have access to real-time information, including specific menus of restaurants. For the most accurate and up-to-date appetizer options at Bella Vista Italian Restaurant, I recommend checking their official website, contacting them directly, or using a reliable food delivery and review platform like Google Maps or Yelp.


In [30]:
query2_with_knowledge = qa.invoke(query2)
query2_without_knowledge = llm.invoke(query2)

print(query2_with_knowledge['result'])
print()

<think>
Okay, so I need to figure out the vegetarian main course options from the given menu. Let me go through the context provided. 

Looking at the main courses, there are three options listed: Spaghetti alla Carbonara, Risotto ai Funghi Porcini, and Lasagna alla Bolognese. 

First, Spaghetti alla Carbonara is made with eggs, cheese, and pancetta. Pancetta is a type of cured meat, so that's not vegetarian. So that's out.

Next, Risotto ai Funghi Porcini. This one uses Arborio rice, porcini mushrooms, white wine, and Parmigiano-Reggiano cheese. I don't see any meat mentioned here, so this seems vegetarian. Plus, the description doesn't include any meat products, so it should be safe.

Then, Lasagna alla Bolognese. The description says it has a meat ragù made with ground beef and pork. That definitely includes meat, so that's not vegetarian.

So the only vegetarian main course is the Risotto ai Funghi Porcini. The price is $26. Let me make sure I didn't miss anything else. The beverag

In [31]:
print(query2_without_knowledge.content)

<think>
Okay, so I need to figure out what vegetarian main course options are available and then describe one in detail, including its price. Hmm, let's start by thinking about common vegetarian main dishes. I know that vegetarian options can vary a lot depending on the cuisine and restaurant, but there are some staples. 

First, I should consider different types of vegetarian main courses. There's pasta, stir-fries, curries, burgers, and maybe some grain bowls or salads. Wait, but the user is asking for a main course, so maybe something more substantial than a salad. 

I remember that eggplant parmesan is a popular vegetarian option. It's an Italian dish that's pretty hearty. Let me think about how it's prepared. It usually involves slices of eggplant that are breaded and fried, then layered with tomato sauce and cheese, and baked. That sounds like a good candidate.

Now, I should describe it in detail. The dish would start with eggplant slices. They're typically breaded, maybe with s